In [11]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt

In [12]:
#select image and rotational offset
path = r"3dTo2d\orthogonal_view_first.png"
img = cv2.imread(path, 0)
dim, _ = img.shape[:2]
box_dim = LR.BOX_DIM
offset = 5
prim_rot = 0
secd_rot = 0


In [13]:
#create back-projection for image
def createBox(img):
  line_coords = LR.createCoordsForLoadedImageFrom3d(img, True, dim)
  world_coords = LR.performBresenham3D(line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  box_rot = LR.rotatePrimSecond(box, prim_rot, secd_rot)
  box_rot = np.where(box_rot > 0.5, 1, 0)
  return box , box_rot

box , box_rot = createBox(img)


In [14]:
#create slightly rotated boxes
rotations = [(offset, 0), (-offset, 0),
             (0, offset), (0, -offset), 
             (offset, offset), (-offset, offset),
             (offset, -offset), (-offset, -offset)]

boxes_rot = [box_rot]
for i in rotations:
  temp_rot = LR.rotatePrimSecond(box_rot, i[0], i[1])
  temp_rot = np.where(temp_rot>0.5,1,0)
  boxes_rot.append(temp_rot)
  print("rotated", i[0],i[1])


rotated 5 0
rotated -5 0
rotated 0 5
rotated 0 -5


In [16]:
#find uncertancy
uncertainty = []

#loop all translated boxes and calculate overlap
box_img = boxes_rot[0]
for b in boxes_rot:
  box_sum = box_img + b
  box_sum = np.where(box_sum == 2, 1, 0)  # save overlap
  uncertainty.append(np.sum(box_sum))

print(uncertainty/np.sum(box_img))


[1.         0.77077268 0.77131635 0.77675186 0.77692726]


In [ ]:
#render box(es)
x, y, z = LR.boxToWorldCorrds(boxes_rot[0], box_dim)


In [ ]:
mlab.points3d(x, y, z,
              mode="cube", color=(1, 0, 0.5), scale_factor=1, opacity=1)
mlab.show()
